## Smart detection with blocks knowledge 

In [1]:
%matplotlib inline
import csv
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import collections
import numpy as np

In [2]:
# Load a snapshot of a network 
G = nx.read_gpickle('trustchain.pickle')

# Eventually consistent Ledger for P2P 


We don't want to use global consensus algorithms (i.e. BFT based on voting, or exchanging a lot of redundant information). Such consensus algorithms are not scalable either in a number of peers or throughput.
An implicit consensus with related peers might work. 


I have studied applications where Trustchain can shine and very few require strong consistency on the events. 
Identity, bandwidth exchange and ledger and ATMs in banks don't have strong consistency. They allow marginal quantifiable error in exchange for availability and scalability.

There is practically no need to use a totally ordered set(Chain) even for one account. We need to look into the partially/casually ordered set. There is no dependency between getting bandwidth from two peers, such events can easily happen in parallel. Using a totally ordered requires and even forces peer to sync on a counter, it is like using a lock, it is easy and does it is the thing, but it is a huge overkill for most of the applications.  

What we want is to quantify the risk double-spending, or to put more generally we want to minimize the ability to hide some information from the other peer. Having near full information is crucial to the decision making the process of the peer. So don't need the full ordering of the event but knowledge completeness.  


To guarantee eventual consistency with quantifiable risk: 
 1. Each peer must share to the network the event 
 
  - If the event is not shared in the network it is impossible to protect/mitigate hiding
  - To save bandwidth the event must be shared in the network with the peers that are interested/subscribed for this information
  - Peer must known at least `f+1` peer that are related to an account `A` - *witnesses*. Peer that had/or have common history with `A`.   
  
 2. Each transaction must be build on top of the lastest state(lastest set of known events)
 
  - This guarantees that block cannot be hidden and all related peers will eventaully converge to the same common state
  - The state is presented with a partially ordered set *PoSet* - this is enough to gurantee convergance, but they are usually much more parallel-friendly. DS to test out: **DAG-CHAIN, Vector(Bloom)Clock, Merkle tree variants**. Each of these data structures are used to compactly accumulate sets of parially ordered sets. 
  
 3. The consistency of the system is formed as following: 
 
   - We need to maintain a system invariant, for example account balance must be positive and etc.
   - Some invariants are easy to maintain in eventual world: For example growing only counter and a low bound
   - We can guarantee invariants by pre-conditions of each transaction
   - Some applications require strict consitency - no invariant violation, but some are less strict on that, we aim for these applications.
   - There several ways to achieve just-enough consistency for invariants: 
       1. Smaller transactions when approaching the invariant bound 
       2. Give the consistency completly, but don't serve/... when invariant is broken


In [ ]:
# Tribler token ledger 
# Creating a Token Economy for Anonymous bandwidth sharing 
# Anonymous network -> well-defined operations 

# Accounting mechanisms 
# Making the accounting mechanisms useful 


In [1]:


def store_transaction(ledger, tx):
    '''
    Store transaction in the local ledger
    '''
    tx['from']
    tx['count']
    tx['to']
    tx['hash']
    tx['prev_hash']
    tx['sig']
    
    

In [ ]:

   A: Bloom filter A increase with operation in B $H_k(B)+=1$
   
   B: Confirm that B received and update it's state $H_k(A)+=1$
   
   


General Transaction format: `from, to, payload(count), hash, signature`


## 0. Store in a Account - Chain with a counter

Store as it stored now - all events realted to account are connected in a totally ordered chain. 

When peer recieves a new transaction with an inclusion proof it can download necessary transaction from local head to current head in the transaction. 

**Inclusion proof:**

1. Each transaction has a chained has based on a continously chaining hash root of a Account Chain 

**Validation:**

- To validate peer will rebuild it's own local chain by donwloading realted transactions.
- Peer constructs chain  from the last head to the current head, if it find some inconsistency it will suspect peer B that it changed the chain / or it was restructured? Possible strategies:
 + Ask other peer about the chain - local quorum consensus 
 + Keep transactions from peer B 
- When peer A sees a counter it must download other transactions from other peer it is able to download from peer B or if it sees that transaction is a confirmation transaction from peer C, it may request state from peer C.
- Some transaction might not be confirmed but they are part of the chain so peer A cannot hide it. 

**Double spending scenario:** 
 
1. Peer A is double spending and peer B and peer C is estimating double spend probability etc.  
- Peer A creates a source block b and another block b' with the same previous hash for both blocks
- This can be detected but no scheme to reconstruct, restructure the chain: 
 + DAG: Continue as a dag, allow multiple previous heads 
 
 
***How to compress the chain?*** 




**Quickly synchronize when you out of sync and detect the chain holes/ difference?**

Transaction in this case includes a counter that show the order in the chain. When peer receives a transactions out of order
it can't apply it before downloading other blocks. This makes other peers wait for the synchronization, 
but decreases double spend probability?   

Double spend will be eventually detected because of the previous layer, 
but it is unclear how to recover from the double spend, as in the chain there must be only one ground true.

### Chain-Lattice: 
Store counters per peer-interactions: 
See `Vector clock`

Store transactions in a chain, but the counter is per-interaction.
 
#### Why?


### 0.1 Chain-DAG modification

A transaction can include multiple links to the transactions. Inclusion of a link means that it is know. 
Each transaction will have multiple previous links - for a merging semantics. 

#### Merge semantics

Transaction includes multiple links from last seen chains





## 1. Store in a vector clock

Suppose that we can limit number of account and which are static. 

$VC_A = {0, 0, 0 \dots 0}$

Transfer to $B --> VC_A(B)+=1$

Each transaction increases a counter of counter-account. Each account update is independent This can be performed indepdently from each other.   

Basically account chain with a counter is partially a vector clock. 

**Inclusion Proof:** 

 1. Full vector clock in a block: all accounts clock 
  - This helps to quickly synchronize for other peers, but a lot redundant information is exchanged.  
 2. Delta in a vector clock. For example, just a simple counter is enough. B: 34 -> 35 
  - When other peer sees this update, it can learn if it is synchrnoized with information regarding to B. 

**Downsides**:
 - *Not scalable in number of accounts*
 - *Big message number to exchange* 
 
**Validation**:
 - Verifiy that vector clock is recent and known, aka can be reconstructed from the local clock
 

### 1.1. Bloom clock modification



## 2. Store in a Merkle Tree

Merkle tree can be used to compress transaction into a merkle root with a state sketch.

Each transaction will include a resulting merkle root. 


When considereing an application you need to think first what need to be consistent: 
 
 Think in invariants: 

# Related work
## Set-reconcilation with cuckoo filters

### One filter per account?

The performance of the filter will degrade with more values inserted. 
To improve performance we need to introduce moving window of the elements, or limit the number of transactions. 

Filter will be filled with transaction id/hash. 

Having a filter and a new transaction one can check quickly if the transacion is inserted in the right time. 
Bloom clock 

## Transaction 

Source transaction: Account $A$ is making a sending bandwith transaction to $B$:
   
   Transaction id: $T_{id}$, update to state $A$ and to state $B$  


Transaction is a state transaction of a Peer A from $T_k: s_k -> s_{k+1}$. 

State can be represented as a Bloom clock? Counting Bloom Filter or Cuckoo filter

Each peer has own clock, event happend can be computed -> $H_k(B)+=1$ for each $H_k$. This will update certain cells in the clock. 

Transactions with different peers can be executed concurrently and later merged. 


